In [ ]:
import datetime as dt
import os

import pysat
import netCDF4

In [ ]:
# Check for pysat data directory
if len(pysat.params['data_dirs']) == 0:
    print('Assigning demo directory.')
    pysat.params['data_dirs'] = './pysatDemo'

In [ ]:
filename = 'demo_test_file_{year:04d}{day:03d}.nc'
date = dt.datetime(2009, 1, 1)

In [ ]:
# Instrument with variety of 1D variables
inst = pysat.Instrument('pysat', 'testing')

# Instrument with a variety of 1D and 2D variables
# inst = pysat.Instrument('pysat', 'testing2D')

# Instrument with xarray data, mixed data dimensipnality.
# inst = pysat.Instrument('pysat', 'testmodel')

# Instrument with xarray data, mixed data dimensipnality.
# inst = pysat.Instrument('pysat', 'testing2D_xarray')

In [ ]:
# Load data
inst.load(date=date, use_header=True)

In [ ]:
# Options when writing files.

# Translate metadata labels to something new.

# Metadata labels in the file may be different than used by Instrument object.
# Default behavior
meta_translation = None
inv_translation = None
export_nan = None

# # Map existing labels to multiple labels in the file

# Made up translation #1
# meta_translation = {inst.meta.labels.units: ['funny_units', 'serious_units'],
#                    inst.meta.labels.fill_val: ['funny_fill', 'fill_serious']}
# inv_translation = {'funny_units': inst.meta.labels.units,
#                    'serious_units': inst.meta.labels.units,
#                    'funny_fill': inst.meta.labels.fill_val,
#                    'fill_serious': inst.meta.labels.fill_val}


# Arbitrary processing of metadata is also supported when writing/loading files.
# See pysat documentation for more on the `meta_processor` keyword.


# Add additional metadata

# `new_label` will only appear in the file for 'mlt' since values for other variables are NaN. To include metadata
# with NaN values, use the `export_nan` keyword. It will, by default, include fill, and the min and max values.
# Note that adding a new metadata type to meta will also add it to `meta.labels`.

# drop_label = 'new_label'
# inst.meta['mlt'] = {drop_label: 1.}
# inst.meta.data

# export_nan = [inst.meta.labels.fill_val, inst.meta.labels.max_val,
#               inst.meta.labels.min_val, inst.meta.labels.drop_label]


# By setting the `drop_meta_labels` keyword, users can prevent metadata information from being loaded.

# Keep all metadata
drop_labels = []

# Drop newly added label
# drop_labels = ['new_label']

In [ ]:
# Add custom information to `inst.meta.header` which is written to file.
inst.meta.header.demo_thang_pysat_style = 'Yes'

In [ ]:
# Write file, but first, format filename.
form_filename = os.path.join(inst.files.data_path, filename.format(year=inst.yr, day=inst.doy))

# Write file using `pysat.utils.io`.
pysat.utils.io.inst_to_netcdf(inst, form_filename, meta_translation=meta_translation, export_nan=export_nan)

In [ ]:
# List global file attributes
with netCDF4.Dataset(form_filename) as data:
    print('Global File Attributes\n')
    for attr in data.ncattrs():
        print('\n', attr, ': ', data.getncattr(attr))

In [ ]:
# List variable metadata
with netCDF4.Dataset(form_filename) as data:
    print('File Variable Attributes\n')
    for var in data.variables.keys():
        print('File Variable: ', var)
        print(''.join(['----------------','-'*len(var)]))
        
        for nc_key in data.variables[var].ncattrs():
            print(nc_key, ': ', data.variables[var].getncattr(nc_key))
            
        print('\n')


In [ ]:
# Load data and and meta using `pysat.utils`
data, meta = pysat.utils.io.load_netcdf(form_filename, pandas_format=inst.pandas_format, 
                                        meta_translation=inv_translation,
                                        drop_meta_labels=drop_labels)

# Print loaded metadata information to Jupyter.
meta.data

In [ ]:
# Print loaded data to Jupyter.
data

In [ ]:
# Load data using pysat.Instrument. Presumes that current `inst` created NetCDF file.
# First, define general pysat.Instrument.
load_inst = pysat.Instrument('pysat', 'netcdf', pandas_format=inst.pandas_format, 
                             update_files=True, file_format=filename, data_dir=inst.files.data_path,
                             meta_translation=inv_translation, drop_meta_labels=drop_labels)

# Load data.
load_inst.load(2009, 1, use_header=True)

# Print to Jupyter.
load_inst.data

In [ ]:
# Print metadata to Jupyter.
load_inst.meta.data